In [220]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 25 20:32:30 2021

@author: Matt Pitz
"""


import user_data as user
import database_functions as dbf
import recommender as rec

def main():
    #collect user inputs (note only works on new user for now
    new_user, user_id, gender, age_group, month, updated_user_df = user.get_user_data()
    
    #collect user goals
    weekly_target, days_to_run, medium_intensity_runs, high_intensity_runs, long_run = user.get_run_plan()
    
    #load db in memory (Move this to server start when website work)
    raw_df = dbf.load_data()
    print('finished loading')
    
    #pass user data to db
    update_user_df = dbf.update_database(new_user, gender, age_group, month, weekly_target, days_to_run, user_id, updated_user_df)
    
    #create dataset with user data for model
    filtered_df = dbf.database_for_recommender(raw_df, update_user_df, gender, age_group, month, days_to_run, weekly_target)
    
    #return recommendations
    recommendations_df = rec.generate_run_ratings(filtered_df,user_id, weekly_target, days_to_run)
    
    #return training plan
    run_schedule = rec.return_run_schedule(recommendations_df, days_to_run, weekly_target, medium_intensity_runs, high_intensity_runs, long_run)
    
    return run_schedule

schedule = main()
schedule
#eof

Are you a new user? (yes/no):  no
Please enter your user id:  99999
How many days did you run in the last 7 days?:  4
What is your gender? (M/F/Other)  M
What is your age?  35
Enter a date you ran in MM/DD/YYYY format:  08/10/2023
How many kilometers did you run?:  10
What was the total run time in HH:MM:SS format?:  00:42:00
Enter a date you ran in MM/DD/YYYY format:  08/11/2023
How many kilometers did you run?:  11
What was the total run time in HH:MM:SS format?:  00:50:00
Enter a date you ran in MM/DD/YYYY format:  08/12/2023
How many kilometers did you run?:  9
What was the total run time in HH:MM:SS format?:  00:38:00
Enter a date you ran in MM/DD/YYYY format:  08/13/2023
How many kilometers did you run?:  15
What was the total run time in HH:MM:SS format?:  01:30:45
How many km do you plan to run this week?  45
How many days do you plan to run?  5
How many medium intensity runs would you like?  0
How many high intensity runs would you like?  0
Would you like a Sunday long run? (y

finished loading


/home/ec2-user/Capstone/App/database_functions.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  user_df['Duration'] = pd.to_datetime(user_df['Duration'])
/home/ec2-user/Capstone/App/database_functions.py:82: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  updated_user_df['Duration'] = pd.to_datetime(updated_user_df['Duration'])


,run_distance,pace
0,10,00:06:30
1,8,00:06:30
2,6,00:06:30
3,18,00:06:30
4,11,00:06:30
5,13,00:06:30
6,7,00:06:30
